https://medium.com/swlh/web-scraping-python-requests-and-beautifulsoup-45d5f48f5a1

https://sparkbyexamples.com/spark/spark-read-text-file-rdd-dataframe/

https://medium.com/swlh/text2sql-in-spark-nlp-converting-natural-language-questions-to-sql-queries-on-scale-6ae9a9061d74

https://www.oreilly.com/library/view/learning-spark-2nd/9781492050032/ch04.html

In [1]:
#!python -m pip install html2text

In [2]:
#!python -m pip install mistletoe

https://github.com/webrecorder/warcio

In [3]:
#!pip install warcio

In [4]:
#!python -m pip install cdx_toolkit

In [5]:
#%load_ext autoreload
#%autoreload 2

Run the cell below to import everything necessary for this analysis.

In [6]:
import json
import sys
import os.path
import re
import logging
import pandas as pd
import requests
import warcio
import mistletoe
import cdx_toolkit
import zlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from warcio import ArchiveIterator
from contextlib import closing
from html2text import HTML2Text
from bs4 import BeautifulSoup
from pathlib import Path
from IPython.display import HTML as HTML_raw, display
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
np.random.seed(0)

# Our corpus

In this analysis, we'll be working with alomost 100k different documents, each containing one single job AD.

These job AD plain text files are contained within the `\c\Users\renjm\Job-Posting-Big-Data\ana\dat\100k_jobADs` subdirectory, contained within the same folder as this analysis. Each job AD is stored in a single file, with files ranging from `1_2021JAN_jobAD.txt` to `99422_2021JAN_jobAD.txt`.

To make it easy to read in all of the documents, use a list comprehension to create a list containing the name of every single job AD plain text file below.

In [7]:
filesList = []
n_0 = 1
n_1 = 100000
for i in range(n_0, n_1):
    filename = './dat/100k_jobADs/'+str(i)+'_2021JAN_jobAD.txt'
    if (os.path.exists(filename)):
        filesList.append(str(i)+'_2021JAN_jobAD.txt')

print(filesList)

['1_2021JAN_jobAD.txt', '11_2021JAN_jobAD.txt', '12_2021JAN_jobAD.txt', '21_2021JAN_jobAD.txt', '22_2021JAN_jobAD.txt', '23_2021JAN_jobAD.txt', '27_2021JAN_jobAD.txt', '28_2021JAN_jobAD.txt', '33_2021JAN_jobAD.txt', '34_2021JAN_jobAD.txt', '39_2021JAN_jobAD.txt', '41_2021JAN_jobAD.txt', '42_2021JAN_jobAD.txt', '45_2021JAN_jobAD.txt', '49_2021JAN_jobAD.txt', '53_2021JAN_jobAD.txt', '54_2021JAN_jobAD.txt', '55_2021JAN_jobAD.txt', '56_2021JAN_jobAD.txt', '62_2021JAN_jobAD.txt', '65_2021JAN_jobAD.txt', '69_2021JAN_jobAD.txt', '74_2021JAN_jobAD.txt', '84_2021JAN_jobAD.txt', '87_2021JAN_jobAD.txt', '91_2021JAN_jobAD.txt', '92_2021JAN_jobAD.txt', '96_2021JAN_jobAD.txt', '108_2021JAN_jobAD.txt', '115_2021JAN_jobAD.txt', '116_2021JAN_jobAD.txt', '117_2021JAN_jobAD.txt', '118_2021JAN_jobAD.txt', '121_2021JAN_jobAD.txt', '124_2021JAN_jobAD.txt', '125_2021JAN_jobAD.txt', '129_2021JAN_jobAD.txt', '131_2021JAN_jobAD.txt', '132_2021JAN_jobAD.txt', '134_2021JAN_jobAD.txt', '135_2021JAN_jobAD.txt', '14

next, create an empty DataFrame called `jobads_df`. As we read in job ads and store and clean them, we'll store them in this DataFrame.

In [8]:
jobads_df = pd.DataFrame()

next, let's import one job AD to see what our text looks like so that we can make sure we clean and tokenize it correctly.

read in and print out the `job description` from `11_2021JAN_jobAD.txt`. Use vanilla Python, no pandas needed.

In [9]:
with open('./dat/100k_jobADs/93148_2021JAN_jobAD.txt') as f:
    test_jobad = f.readlines()
    print(test_jobad)

['Hello  Guest! \n', ' Login \n', '       \n', ' Trades Sign UpJob Seeker Sign Up   | \n', '   \n', ' | \n', ' | \n', ' 0 \n', '   \n', ' \n', '   \n', ' Search jobs by... \n', ' \n', ' \n', ' Advanced Search \n', ' \n', ' \n', ' Job Location Search \n', ' \n', ' \n', ' Job Category Search \n', ' \n', ' \n', ' Job Reference Search \n', ' \n', ' \n', ' Home \n', ' \n', ' \n', ' Post a Job \n', ' \n', ' \n', ' Find a Job \n', ' \n', ' \n', ' Business Directory \n', ' \n', ' \n', ' Homeowner Sign UpJob Poster Sign Up \n', ' \n', ' \n', ' Help \n', ' \n', ' \n', ' \n', ' Job Seeker Registration \n', ' \n', ' \n', ' Personal Details \n', ' \n', ' Indicates Mandatory Field \n', ' \n', ' Job Seeker Registration \n', ' Please complete the registration form below \n', ' \n', ' Email Address: \n', ' \n', ' Email Address \n', ' Please enter a valid UK email address \n', ' \n', ' Enter Profile Name:  (visible to the public) \n', ' \n', ' User Name \n', ' Please enter a User Name which can be used 

## Tokenizing our Data

Before we can create a bag of words or vectorize each document, we need to clean it up and split each job AD into an array of individual words. Computers are very particular about strings. If we tokenized our data in its current state, we would run into the following problems:

* Counting things that aren't actually words. 
* Punctuation and capitalization would mess up our word counts. To the Python interpreter, `love`, `Love`, `Love?`, and `Love\n` are all unique words, and would all be counted separately. We need to remove punctuation and capitalization, so that all words will be counted correctly.

Tokenization is pretty tedious if we handle it manually, and would probably make use of `regular expressions`. In order to keep this lab moving, we'll use a library function to clean and tokenize our data so that we can move onto vectorization.

Tokenization is a required task for just about any Natural Language Processing (NLP) task, so great industry-standard tools exist to tokenize things for us, so that we can spend our time on more important tasks without getting bogged down hunting every special symbol or punctuation in a massive dataset. For this analysis, we'll make use of the tokenizer in the amazing `nltk` library, which is short for *Natural Language Tool Kit*.

**NOTE**: `NLTK` requires extra installation methods to be run the first time certain methods are used. If `nltk` throws you an error about needing to install additional packages, follow the instructions in the error message to install the dependencies, and then re-run the cell.

Before we tokenize our job AD plain text files, we'll do only a small manual bit of cleaning. In the cell below, a function that allows us to make every word lowercase, remove newline characters `\n`, and all the following punctuation marks: `",.'?!"` is developed.

Furthermore, this function that not only takes in job AD plain text files having erratic symbols removed but also joins all of the lines into a single string. To sum up, the function, `tokenize_jobAD()`, is utilized to get a fully tokenized version of the job AD. We can test this function on `jobad` to ensure that the function works.

In [10]:
subs = [('.',''),("'",""),('"',''),(',',''),('?',''),('!',''),('\n',''),('\t',''),('\r',''),('|',''),('/',' '),
        ('-',''),('(c)',''),('*',''),('(',''),(')',''),('&',''),(':',''),(';',''),('[',''),(']',''),('\\',''),
        ('â„¢',''),('%',''),('â£',''),('>',''),('<','')]

def tokenize_jobAD(jobad):
    cleaned_jobad = []
    for line in jobad:
        for old, new in subs:
            line = line.replace(old, new).lower()
        cleaned_jobad.append(line)

    tokenized_jobad = ' '.join(cleaned_jobad).split()
    
    return tokenized_jobad

In [11]:
tokenized_test_jobad = tokenize_jobAD(test_jobad)
print(len(tokenized_test_jobad))
print(tokenized_test_jobad[:100000])

47682


['hello', 'guest', 'login', 'trades', 'sign', 'upjob', 'seeker', 'sign', 'up', '0', 'search', 'jobs', 'by', 'advanced', 'search', 'job', 'location', 'search', 'job', 'category', 'search', 'job', 'reference', 'search', 'home', 'post', 'a', 'job', 'find', 'a', 'job', 'business', 'directory', 'homeowner', 'sign', 'upjob', 'poster', 'sign', 'up', 'help', 'job', 'seeker', 'registration', 'personal', 'details', 'indicates', 'mandatory', 'field', 'job', 'seeker', 'registration', 'please', 'complete', 'the', 'registration', 'form', 'below', 'email', 'address', 'email', 'address', 'please', 'enter', 'a', 'valid', 'uk', 'email', 'address', 'enter', 'profile', 'name', 'visible', 'to', 'the', 'public', 'user', 'name', 'please', 'enter', 'a', 'user', 'name', 'which', 'can', 'be', 'used', 'to', 'locate', 'you', 'on', 'worklocally', 'the', 'user', 'name', 'must', 'be', 'between', '6', 'and', '16', 'characters', 'and', 'must', 'not', 'be', 'your', 'company', 'name', 'your', 'email', 'address', 'or', '

Now that we can tokenize our job AD plain text file, we can move onto vectorization.

## Count Vectorization

Machine Learning algorithms don't understand strings. However, they do understand math, which means they understand vectors and matrices. By Vectorizing the text, we just convert the entire text into a vector, where each element in the vector represents a different word. The vector is the length of the entire vocabulary -- usually, every word that occurs in the English language, or at least every word that appears in our corpus. Any given sentence can then be represented as a vector where all the vector is 1 (or some other value) for each time that word appears in the sentence.

`Count Vectorization` allows us to represent a sentence as a vector, with each element in the vector corresponding to how many times that word is used. Notice that when we vectorize a sentence this way, we lose the order that the words were in. This is the `Bag of Words` approach

In the cell below, create a function that takes in a tokenized, cleaned job AD plain text file and returns a count vectorized representation of it as a Python dictionary (aka: *Sparse Representation*). Add in an optional parameter called vocab that defaults to None. This way, if we are using a vocabulary that contains words not seen in the song, we can still use this function by passing it into the vocab parameter.

In [12]:
def count_vectorize(jobad, vocab=None):
    if vocab:
        unique_words = vocab
    else:
        unique_words = list(set(jobad))
    
    jobad_dict = {i:0 for i in unique_words}
    
    for word in jobad:
        jobad_dict[word] += 1
    
    return jobad_dict

In [13]:
test_vectorized = count_vectorize(tokenized_test_jobad)
print(test_vectorized)

{'putting': 3, 'then': 35, 'permit': 6, 'deactivated': 1, 'center': 1, '42': 1, 'piece': 18, 'sorry': 23, 'geographical': 7, 'correspondence': 1, 'sign': 5, 'approved': 5, 'on': 542, 'pursuant': 1, 'cases': 3, 'pays': 1, 'upgraded': 1, 'code': 4, 'legislative': 1, 'cm8': 3, 'advertisements': 4, 'commercial': 78, 'provisions': 1, 'verifications': 1, 'verifiedif': 1, 'success': 1, 'submit': 48, 'within': 60, 'disclosure': 3, 'down': 14, 'according': 2, 'arrangements': 1, 'opinion': 1, 'securing': 2, 'assist': 1, '1024kb': 1, 'suspend': 4, 'anable': 1, 'worst': 1, 'registered': 76, 'term': 2, 'questions': 12, 'habits': 2, 'general': 8, 'price': 5, 'access': 60, 'stated': 2, 'provide': 113, 'max': 5, 'prompt': 1, 'misrepresentation': 1, 'unread': 1, 'depending': 3, 'trial': 1, '1011': 1, 'distinguish': 1, 'previous': 18, 'thoroughly': 1, 'worklocally': 552, 'uptodate': 6, 'restrict': 4, 'skill': 9, 'easier': 6, 'materials': 2, 'medium': 2, 'alphanumerics': 1, '197': 1, 'not': 295, 'control

We've just successfully vectorized the first job AD plain text document! Now, let's look at a more advanced type of vectorization, TF-IDF!

In [14]:
statesList = ["United States of America","USA","AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS",
              "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA",
              "RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC","GU","MH","MP","PR","VI"]
itList = ["engineer","Engineer","scientist","Scientist","developer","Developer","Analyst","analyst","Operator","operator"]
qaList = ["Qualification","Qualifications","qualification","qualifications","Certification","Certifications","certification",
          "certifications"]

check this text job AD text file to ensure it contains any key word(s) in `qaList`

In [17]:
def qaCount(tokenized_jobad):
    counter_qa = 0
    for qa in qaList:
        if qa in tokenized_jobad.keys():
            print(qa)
            counter_qa += 1
    return counter_qa

In [18]:
qaNums_in_test_vectorized = qaCount(test_vectorized)
print(qaNums_in_test_vectorized)

qualification
qualifications
certifications
3
